Import the pynrrd package supplementary class. Load the image and mask.

In [2]:
import nrrd
from scripts.ImageSliceViewer3D import ImageSliceViewer3D as isv 

# paths to image and corresponding mask
im_path = 'data/images/LIDC-IDRI-0001.nrrd'
msk_path = 'data/masks/LIDC-IDRI-0001.nrrd'

im_V,im_d = nrrd.read(im_path)
msk_V,msk_d = nrrd.read(msk_path)

In [7]:
isv(im_V)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

Use `skimage.measure`'s `label` method to locate the radiologist-defined mask. Recall that each pixel in the mask was agreed to be in the lesion with minimum 50% agreement between 4 radiologists.

In [3]:
from skimage.measure import label,regionprops

# label the mask into connected regions
nodule_labels,num_labels = label(msk_V,return_num=True)
nodule_props = regionprops(nodule_labels)

# the number of nodules in nodule_props corresponds to the number of connected regions
print('There are {} nodule(s) in this image.'.format(num_labels))     

There are 1 nodule(s) in this image.


In [4]:
# isolate the centroid of the nodule -- this will be our marker for the watershed algorithm
nodule_props[0].centroid
centroid = [round(i) for i in nodule_props[0].centroid] 
centroid

[367, 316, 90]

In [5]:
regionprops?

nodule_props[0].bbox  # to help construct scale bar

(343, 299, 86, 388, 336, 94)

In [6]:
from skimage.segmentation import watershed

# watershed?

In [8]:
from scipy.ndimage import median_filter as median
from skimage.filters import gaussian
import numpy as np

# denoise image
# denoised = median(im_V,3)
denoised = gaussian(im_V)

print(denoised.shape)
isv(denoised)
print(np.max(denoised))

(512, 512, 133)


interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

0.09368016247167366


In [9]:
from skimage.morphology import erosion, dilation, ball

# try some funky stuff
max_image = dilation(denoised,ball(2))
min_image = erosion(denoised,ball(2))

gradient_image = max_image - min_image

isv(gradient_image)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [10]:
from skimage.measure import label
from skimage.morphology import binary_opening

threshold_gradient = gradient_image < 0.01
isv(threshold_gradient)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [11]:
markers = label(threshold_gradient)
isv(markers,cmap='jet')

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [12]:
# process the watershed
labels = watershed(gradient_image, markers)

In [13]:
isv(labels,cmap='jet')

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [14]:
labels.shape
centroid
water_mask = labels == labels[centroid[0],centroid[1],centroid[2]]

In [15]:
isv(water_mask)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [16]:
import numpy as np

iou_mask = np.count_nonzero(np.logical_and(water_mask,msk_V))/np.count_nonzero(np.logical_or(water_mask,msk_V))
iou_back = np.count_nonzero(np.logical_and(~water_mask,~msk_V))/np.count_nonzero(np.logical_or(~water_mask,~msk_V))

In [17]:
iou_mask     # <-- IoU for the mask

0.7816774658027358

In [18]:
iou_back

0.9998717630716195

In [19]:
(iou_mask + iou_back) / 2    # <-- mean IoU

0.8907746144371776

<div class="alert alert-block alert-info">
<b>FYI:</b> Anything below this block is strictly experimental.
</div>

In [ ]:
from skimage.measure import regionprops
from skimage.morphology import binary_opening

mask_props = label(water_mask)
mask_attr = regionprops(mask_props)
# mask_attr[0].major_axis_length

radius = mask_attr[0].major_axis_length / 2 * 0.03

water_mask_clean = binary_opening(water_mask,ball(radius))

In [ ]:
isv(water_mask_clean)      # <-- doesn't seem to work well

In [ ]:
from skimage.segmentation import morphological_chan_vese

mask2 = morphological_chan_vese(denoised, 50) # <-- takes a LONG time to run: gets one devil horn but doesn't recognise the separation between the melted chocolate and the pleura

In [ ]:
isv(mask2)